In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,578 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,228 kB]

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import month, year

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""Select date_built, round(Avg(price),2)
          from home_sales 
          where bedrooms=4 
          group by date_built 
          order by date_built asc""").show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           296800.75|
|      2011|            302141.9|
|      2012|           298233.42|
|      2013|           299999.39|
|      2014|           299073.89|
|      2015|           307908.86|
|      2016|           296050.24|
|      2017|           296576.69|
+----------+--------------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""Select date_built, round(Avg(price),2)
          from home_sales 
          where bedrooms=3 AND bathrooms=3
          group by date_built 
          order by date_built asc""").show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           292859.62|
|      2011|           291117.47|
|      2012|           293683.19|
|      2013|           295962.27|
|      2014|           290852.27|
|      2015|            288770.3|
|      2016|           290555.07|
|      2017|           292676.79|
+----------+--------------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""Select date_built, round(Avg(price),2)
          from home_sales 
          where bedrooms=3 AND bathrooms=3
          AND floors=2 AND sqft_living>=2
          group by date_built 
          order by date_built asc""").show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           280447.23|
|      2011|           281413.45|
|      2012|           295858.68|
|      2013|           295142.13|
|      2014|           294195.13|
|      2015|           291788.36|
|      2016|            287812.6|
|      2017|           282026.59|
+----------+--------------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""Select view, round(Avg(price),2)
          from home_sales 
          where price>=350000
          group by view
          order by view asc""").show()
run_time = (time.time() - start_time)
print(f'The total run time was {run_time:.2f} seconds')

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|  10|           401868.43|
| 100|           1026669.5|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
|   2|           397389.25|
|  20|           399522.81|
|  21|           399758.88|
|  22|           402022.68|
|  23|           403411.92|
|  24|           400284.92|
|  25|           401298.69|
+----+--------------------+
only showing top 20 rows

The total run time was 0.66 seconds


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""Select view, round(Avg(price),2)
          from home_sales 
          where price>=350000
          group by view
          order by view asc""").show()

run_time_cached = (time.time() - start_time)
print(f'The total run time was {run_time:.2f} seconds')
print(f'The total run time (cached) was {run_time_cached:.2f} seconds')

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|  10|           401868.43|
| 100|           1026669.5|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
|   2|           397389.25|
|  20|           399522.81|
|  21|           399758.88|
|  22|           402022.68|
|  23|           403411.92|
|  24|           400284.92|
|  25|           401298.69|
+----+--------------------+
only showing top 20 rows

The total run time was 0.66 seconds
The total run time (cached) was 0.46 seconds


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [13]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('home_sales_partitioned')

In [14]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""Select view, round(Avg(price),2)
          from p_home_sales 
          where price>=350000
          group by view
          order by view asc""").show()

run_time_p = (time.time() - start_time)
print(f'The total run time was {run_time:.2f} seconds')
print(f'The total run time (cached) was {run_time_cached:.2f} seconds')
print(f'The total run time (cached/partioned) was {run_time_p:.2f} seconds')

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|  10|           401868.43|
| 100|           1026669.5|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
|   2|           397389.25|
|  20|           399522.81|
|  21|           399758.88|
|  22|           402022.68|
|  23|           403411.92|
|  24|           400284.92|
|  25|           401298.69|
+----+--------------------+
only showing top 20 rows

The total run time was 0.66 seconds
The total run time (cached) was 0.46 seconds
The total run time (cached/partioned) was 1.47 seconds


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False